## Paper Pipeline

In [ ]:
import os
import pandas as pd
import numpy as np
import warnings
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from pyswarm import pso
from joblib import Parallel, delayed
from multiprocessing import Pool
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings("ignore")

In [ ]:
BIN_SIZE = 50 # number of bins for binned sequencing data
NUM_CPS = 2 # number of control points per pattern
MP_CUTOFF = 0.3 # min pattern match threshold to count it

ETA = 0.2 # learning rate for XGBoost
NROUNDS = 5 # number of boosting rounds

PSO_SWARMSIZE = 5 # number of particles in the PSO swarm
PSO_MAXITER = 4 # max number of PSO iterations

SEQ_BASE = "Converted_binned_sequencing_data"
LABEL_BASE = "GE_Labels"
SPLIT_BASE = "Gene_Splits"
OUTPUT_BASE = "PatternChrome_Results_SmartBalanced"
os.makedirs(OUTPUT_BASE, exist_ok=True)

In [ ]:
# loads histone modification data for a cell line
def load_histone_data(cell):
    mods = ["H3K4me1", "H3K4me3", "H3K9me3", "H3K27me3", "H3K36me3"]
    return {
        mod: pd.read_csv(os.path.join(SEQ_BASE, cell, f"{mod}_{BIN_SIZE}bp_bins.csv"), index_col=0)
        for mod in mods
    }

# loads GE labels and train/val/test gene splits for a cell line
def load_labels_and_splits(cell):
    y_train = pd.read_csv(os.path.join(LABEL_BASE, cell, "train_labels.csv"), index_col="gene_id").squeeze()
    y_val   = pd.read_csv(os.path.join(LABEL_BASE, cell, "validation_labels.csv"), index_col="gene_id").squeeze()
    y_test  = pd.read_csv(os.path.join(LABEL_BASE, cell, "test_labels.csv"), index_col="gene_id").squeeze()
    g_train = pd.read_csv(os.path.join(SPLIT_BASE, cell, "train_genes.csv"), header=None).squeeze().tolist()
    g_val   = pd.read_csv(os.path.join(SPLIT_BASE, cell, "validation_genes.csv"), header=None).squeeze().tolist()
    g_test  = pd.read_csv(os.path.join(SPLIT_BASE, cell, "test_genes.csv"), header=None).squeeze().tolist()
    return (y_train, y_val, y_test), (g_train, g_val, g_test)

# counts how many times a pattern occurs in each row with corr > threshold
def pattern_frequency(df, pattern, threshold, n_jobs=-1):
    width = len(pattern)
    
    def count_matches(values):
        count = 0
        for i in range(len(values) - width + 1):
            segment = values[i:i + width]
            if np.std(segment) == 0 or np.std(pattern) == 0:
                continue
            corr = np.corrcoef(segment, pattern)[0, 1]
            if not np.isnan(corr) and corr > threshold:
                count += 1
        return count

    rows = df.to_numpy()
    return np.array(Parallel(n_jobs=n_jobs)(delayed(count_matches)(row) for row in rows))

# returns negative AUC for given pattern and region – used by PSO
def objective_function(params, hm_train, y, threshold):
    start, end = int(params[0]), int(params[1])
    if end <= start or (end - start) < NUM_CPS:
        return 0  # invalid region
    pattern = params[2:]
    region = hm_train.iloc[:, start:end]
    freq = pattern_frequency(region, pattern, threshold, n_jobs=-1)
    if np.all(freq == 0): return 0
    model = XGBClassifier(n_estimators=NROUNDS, eta=ETA, use_label_encoder=False, eval_metric="auc", verbosity=0)
    model.fit(freq.reshape(-1, 1), y)
    return -roc_auc_score(y, model.predict_proba(freq.reshape(-1, 1))[:, 1])  # we minimize this

In [ ]:
# runs full pipeline for a given cell line: pattern extraction, feature selection, hyperparameter tuning, evaluation
def process_cell(cell):
    cell_out = os.path.join(OUTPUT_BASE, cell)
    os.makedirs(cell_out, exist_ok=True)

    try:
        histones = load_histone_data(cell)
        (y_train, y_val, y_test), (g_train, g_val, g_test) = load_labels_and_splits(cell)
    except:
        return None  # skip if data loading fails

    common_train = list(set(g_train) & set(y_train.index))
    common_val = list(set(g_val) & set(y_val.index))
    common_test = list(set(g_test) & set(y_test.index))
    train_df = pd.DataFrame({"GE": y_train.loc[common_train]})
    val_df = pd.DataFrame({"GE": y_val.loc[common_val]})
    test_df = pd.DataFrame({"GE": y_test.loc[common_test]})

    patterns_info = []

    # === 2. Pattern extraction using PSO ===
    for idx in range(NUM_CPS):
        best_auc = -np.inf
        best_entry = None

        for mod_name, hm_data in histones.items():
            try:
                hm_train = hm_data.loc[common_train]
                lb = [0, NUM_CPS] + [0] * NUM_CPS
                ub = [hm_train.shape[1], hm_train.shape[1]] + [1] * NUM_CPS
                args = (hm_train, y_train.loc[common_train], MP_CUTOFF)

                best_params, _ = pso(objective_function, lb, ub, args=args,
                                     swarmsize=PSO_SWARMSIZE, maxiter=PSO_MAXITER, debug=False)
                pattern = best_params[2:]
                start, end = int(best_params[0]), int(best_params[1])
                region = hm_train.iloc[:, start:end]
                freq = pattern_frequency(region, pattern, MP_CUTOFF, n_jobs=-1)

                if np.all(freq == 0):
                    continue

                model = XGBClassifier(n_estimators=NROUNDS, eta=ETA, eval_metric="auc", use_label_encoder=False, verbosity=0)
                model.fit(freq.reshape(-1, 1), y_train.loc[common_train])
                auc = roc_auc_score(y_train.loc[common_train], model.predict_proba(freq.reshape(-1, 1))[:, 1])

                if auc > best_auc:
                    best_entry = {"pattern": pattern, "start": start, "end": end, "freq": freq, "mod": mod_name}
                    best_auc = auc
            except:
                continue

        # add best pattern as new feature if found
        if best_entry:
            pat_name = f"Pattern_{len(train_df.columns)}"
            train_df[pat_name] = best_entry["freq"]
            val_df[pat_name] = pattern_frequency(
                histones[best_entry["mod"]].loc[common_val].iloc[:, best_entry["start"]:best_entry["end"]],
                best_entry["pattern"], MP_CUTOFF, n_jobs=-1
            )
            test_df[pat_name] = pattern_frequency(
                histones[best_entry["mod"]].loc[common_test].iloc[:, best_entry["start"]:best_entry["end"]],
                best_entry["pattern"], MP_CUTOFF, n_jobs=-1
            )
            patterns_info.append({
                "Pattern": pat_name, "HM": best_entry["mod"],
                "Start": best_entry["start"], "End": best_entry["end"], "Width": len(best_entry["pattern"]),
                **{f"Point_{i}": p for i, p in enumerate(best_entry["pattern"])}
            })

    # skip if no features were extracted
    features = list(train_df.columns[1:])
    if not features:
        return None

    # === 3. Feature selection via backward elimination ===
    model = XGBClassifier(n_estimators=NROUNDS, eta=ETA, eval_metric="auc", use_label_encoder=False)
    model.fit(train_df[features], train_df["GE"])
    best_auc = roc_auc_score(val_df["GE"], model.predict_proba(val_df[features])[:, 1])

    for feature in reversed(features):
        reduced = features.copy()
        reduced.remove(feature)
        if not reduced:
            continue
        model.fit(train_df[reduced], train_df["GE"])
        auc = roc_auc_score(val_df["GE"], model.predict_proba(val_df[reduced])[:, 1])
        if auc >= best_auc:
            features = reduced
            best_auc = auc

    # save filtered feature matrix
    train_df[["GE"] + features].to_csv(os.path.join(cell_out, "features_selected.csv"))

    # === 4. Hyperparameter tuning using PSO ===
    def hp_objective(params, X, y, X_val, y_val):
        try:
            m = XGBClassifier(
                n_estimators=int(params[0]), eta=params[1], gamma=params[2],
                max_depth=int(params[3]), reg_lambda=params[4], reg_alpha=params[5],
                min_child_weight=params[6], subsample=params[7],
                use_label_encoder=False, eval_metric="auc", verbosity=0
            )
            m.fit(X, y, eval_set=[(X_val, y_val)], early_stopping_rounds=15, verbose=False)
            return -roc_auc_score(y_val, m.predict_proba(X_val)[:, 1])
        except:
            return 1

    hp_lb = [100, 0.001, 0.0, 2, 0.0, 0.0, 0.0, 0.1]
    hp_ub = [1500, 0.5, 20.0, 20, 10.0, 10.0, 50.0, 1.0]

    best_hp, best_val_score = pso(hp_objective, hp_lb, hp_ub,
                                   args=(train_df[features], train_df["GE"], val_df[features], val_df["GE"]),
                                   swarmsize=PSO_SWARMSIZE, maxiter=PSO_MAXITER)

    # === 5. Final training and evaluation on test set ===
    final_model = XGBClassifier(
        n_estimators=int(best_hp[0]), eta=best_hp[1], gamma=best_hp[2],
        max_depth=int(best_hp[3]), reg_lambda=best_hp[4], reg_alpha=best_hp[5],
        min_child_weight=best_hp[6], subsample=best_hp[7],
        use_label_encoder=False, eval_metric="auc", verbosity=0,
        early_stopping_rounds=15
    )

    final_model.fit(
        train_df[features], train_df["GE"],
        eval_set=[(val_df[features], val_df["GE"])]
    )

    test_auc = roc_auc_score(test_df["GE"], final_model.predict_proba(test_df[features])[:, 1])

    # === 6. Save final results ===
    result = {
        "Cell": cell,
        "Test_AUC": round(test_auc, 5),
        "Validation_AUC": round(abs(best_val_score), 5),
        **{k: round(v, 5) for k, v in zip(
            ["n_estimators", "eta", "gamma", "max_depth", "lambda", "alpha", "min_child_weight", "subsample"],
            best_hp
        )}
    }

    pd.DataFrame([result]).to_csv(os.path.join(cell_out, "final_results_with_hyperparams_tuned.csv"), index=False)

    return result


In [ ]:
results = []

# get all available cell line folders
all_cells = [d for d in os.listdir(SEQ_BASE) if os.path.isdir(os.path.join(SEQ_BASE, d))]
all_cells.sort()

# run full pipeline for each cell line
for cell in all_cells:
    result = process_cell(cell)
    if result:
        results.append(result)

summary_path = os.path.join(OUTPUT_BASE, "test_results.csv")
pd.DataFrame(results).to_csv(summary_path, index=False)

## Report Visualisations

In [ ]:
custom_palette = {
    "PatternChrome (paper)": "#1f77b4",
    "PatternChrome (ours)": "#ff7f0e",
    "PatternChrome (tuned)": "#9467bd",
    "ShallowChrome": "#2ca02c",
    "DeepChrome": "#d62728"
}

performance_df = pd.read_csv("PatternChrome_Results/performance_Metrics.csv")
test_results_df = pd.read_csv("PatternChrome_Results/test_results.csv")
test_results_tuned_df = pd.read_csv("PatternChrome_Results_SmartBalanced/test_results.csv")

cell_lines = performance_df["cell_line"].tolist()

pattern_paper_auc = performance_df["AUC"].tolist()
pattern_ours_auc = test_results_df.set_index("Cell").loc[cell_lines]["Test_AUC"].tolist()
pattern_tuned_auc = test_results_tuned_df.set_index("Cell").loc[cell_lines]["Test_AUC"].tolist()

deepchrome = [0.77,0.81,0.8,0.82,0.77,0.76,0.8,0.79,0.79,0.77,0.8,0.81,0.81,0.83,0.83,0.8,0.8,0.8,0.83,0.89,0.9,0.84,0.9,0.84,0.8,0.76,0.8,0.76,0.73,0.74,0.78,0.71,0.72,0.74,
              0.73,0.82,0.72,0.76,0.74,0.9,0.79,0.78,0.76,0.74,0.69,0.73,0.83,0.91,0.92,0.85,0.83,0.84,0.83,0.92,0.83,0.83]
shallowchrome = [0.878,0.879,0.884,0.872,0.887,0.874,0.898,0.891,0.885,0.880,0.868,0.887,0.891,0.891,0.898,0.905,
                 0.893,0.888,0.901,0.891,0.882,0.893,0.876,0.891,0.883,0.804,0.850,0.85756,0.81386,0.84087,0.86323,
                 0.85549,0.86363,0.83924,0.82661,0.87473,0.84089,0.84009,0.8425,0.88465,0.84114,0.85575,0.84818,
                 0.83271,0.83361,0.85408,0.89235,0.90511,0.913,0.90319,0.89433,0.89197,0.88881,0.91957,0.89446,0.89072]

df = pd.DataFrame({
    "cell_line": cell_lines * 5,
    "AUC": pattern_paper_auc + pattern_ours_auc + pattern_tuned_auc + shallowchrome + deepchrome,
    "Model": ["PatternChrome (paper)"] * 56 +
             ["PatternChrome (ours)"] * 56 +
             ["PatternChrome (tuned)"] * 56 +
             ["ShallowChrome"] * 56 +
             ["DeepChrome"] * 56
})

# Violin plot
plt.figure(figsize=(12, 6))
sns.violinplot(data=df, x="Model", y="AUC", palette=custom_palette, inner="box")
sns.stripplot(data=df, x="Model", y="AUC", color="black", size=2, alpha=0.5, jitter=True)
plt.ylim(0.6, 1.0)
plt.title("AUC Distribution per Model")
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

# Boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x="Model", y="AUC", palette=custom_palette)
sns.pointplot(data=df, x="Model", y="AUC", estimator=np.mean, color='black', markers='D', join=False, ci=None)
plt.title("Boxplot with Mean Marker per Model")
plt.ylim(0.6, 1.0)
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

# Mean and standard deviation
summary = df.groupby("Model")["AUC"].agg(["mean", "std"]).reset_index()
summary.plot(
    kind='bar',
    x='Model',
    y='mean',
    yerr='std',
    legend=False,
    figsize=(10, 6),
    color=[custom_palette[m] for m in summary["Model"]]
)
plt.title("Mean AUC with Standard Deviation")
plt.ylabel("Mean AUC")
plt.ylim(0.6, 1.0)
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

# Lineplot
pivot_df = df.pivot(index='cell_line', columns='Model', values='AUC').reset_index()
melted_line = pd.melt(pivot_df, id_vars='cell_line', var_name='Model', value_name='AUC')

plt.figure(figsize=(14, 6))
sns.lineplot(data=melted_line, x="cell_line", y="AUC", hue="Model", style="Model", markers=True, dashes=False, palette=custom_palette)
plt.xticks(rotation=90)
plt.title("AUC per Cell Line for Each Model")
plt.ylim(0.6, 1.0)
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

# Barplot with min, max and mean AUC per model
summary_minmaxavg = df.groupby("Model")["AUC"].agg(["mean", "min", "max"]).reset_index()
summary_melted = pd.melt(summary_minmaxavg, id_vars="Model", var_name="Metric", value_name="AUC")

plt.figure(figsize=(10, 6))
sns.barplot(data=summary_melted, x="Model", y="AUC", hue="Metric", palette="Set2")
plt.title("Min, Max, and Mean AUC per Model")
plt.ylabel("AUC Score")
plt.ylim(0.6, 1.0)
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()